![](mc_logo.png)

## Adding Assets to Maintenance Program
- **STEP 1** 
    - Add all location types in order (AREA -> SUBAREA / SKID / PIPING, -> SUBAREA / SKID)
- **STEP 2** 
    - Pull tags from .txt/.csv files
- **STEP 3** 
    - Add new tag classifications to configs dict
- **STEP 3** 
    - _get all existing assets from P&ID page and added suffix
- **STEP 4** 
    - _post all new assets
- **STEP 5**
    - Transfer pertinent info from "_old" assets to new assets
- **STEP 6**
    - Rearrange assets in program

In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
import copy
import pandas as pd
import toml
from pathlib import Path

from src._requests import _get, _post, _put

True


#### *Change this file path to reflect page P&ID page:*

In [254]:
_file = pd.read_csv('', header = None)

### *Area Configs:*
<p style="color:red;">- Convert to .toml or .json</p>

In [134]:
area_configs = {'Flare Area':{'Name':'Flare Area','Vicinity':'PID-1800','Subarea':None,'Skid':None,'Piping':{'Name':'Flare Area Piping','Vicinity':'PID-1800'},'Compression':None}, 
                'Fuel Gas Area':{'Name':'Fuel Gas Area','Vicinity':'PID-1800','Subarea':None,'Skid':[{'Name':'C-8701 PVRU Compressor Skid','Vicinity':'PID-1803'}],'Piping':{'Name':'Fuel Gas Area Piping','Vicinity':'PID-1800'},'Compression':[{'Skid':'C-8701 PVRU Compressor Skid','Internals':[{'Name':'C-8701 Module Compressor','Vicinity':'PID-1803'}]}]}, 
                'H2S Treatment Area':{'Name':'H2S Treatment Area','Vicinity':'PID-1300','Subarea':[{'Name':'Sweetening Skid & Separators','Vicinity':'PID-1302A/B'}],'Skid':None,'Piping':{'Name':'H2S Treatment Area Piping','Vicinity':'PID-1300'},'Compression':None}, 
                'Inlet Area':{'Name':'Inlet Area','Vicinity':'PID-1100','Subarea':[{'Name':'Flash Gas Area','Vicinity':'PID-1103/4'},{'Name':'Inlet Pig Receiver Area','Vicinity':'PID-1101A'},{'Name':'Inlet Slug Catcher Area','Vicinity':'PID-1101B'},{'Name':'Low Pressure Separator Area','Vicinity':'PID-1102'}],'Skid':[{'Name':'C-1701 Flash Gas Compressor Skid','Vicinity':'PID-1103'}],'Piping':{'Name':'Inlet Area Piping','Vicinity':'PID-1100'},'Compression':[{'Skid':'C-1701 Flash Gas Compressor Skid','Internals':[{'Name':'C-1701 Drive Train & Major Equipment','Vicinity':'PID-1103D'},{'Name':'C-1701 Lube Oil System','Vicinity':'PID-1103C'},{'Name':'C-1701 Module Compressor','Vicinity':'PID-1103B'},{'Name':'C-1701 Suction Scrubber','Vicinity':'PID-1103A'}]}]}, 
                'Inlet Compression Area':{'Name':'Inlet Compression Area','Vicinity':'PID-1200','Subarea':None,'Skid':[{'Name':'C-2701 Inlet Compressor Skid','Vicinity':'PID-1201'},{'Name':'C-2702 Inlet Compressor Skid','Vicinity':'PID-1202'},{'Name':'C-2703 Inlet Compressor Skid','Vicinity':'PID-1203'},{'Name':'C-2704 Inlet Compressor Skid','Vicinity':'PID-1204'},{'Name':'C-2705 Inlet Compressor Skid','Vicinity':'PID-1205'}],'Piping':{'Name':'Inlet Compressor Area Piping','Vicinity':'PID-1200'},'Compression':[{'Skid':'C-2703 Inlet Compressor Skid','Internals':[{'Name':'C-2703 1st Stage Equipment', 'Vicinity':'PID-1203A'}, {'Name':'C-2703 2nd Stage Equipment', 'Vicinity':'PID-1203B'},{'Name':'C-2703 3rd Stage Equipment', 'Vicinity':'PID-1203C'},{'Name':'C-2703 AJW Cooling System', 'Vicinity':'PID-1203K'},{'Name':'C-2703 Compressor Lubricant Equipment', 'Vicinity':'PID-1203E'},{'Name':'C-2703 Cooler Equipment', 'Vicinity':'PID-1203D'},{'Name':'C-2703 Cooler Equipment', 'Vicinity':'PID-1203J'},{'Name':'C-2703 Engine Lubricant Equipment', 'Vicinity':'PID-1203G'},{'Name':'C-2703 Start/Fuel Gas System', 'Vicinity':'PID-1203H'}]},{'Skid':'C-2704 Inlet Compressor Skid','Internals':[{'Name':'C-2704 1st Stage Equipment', 'Vicinity':'PID-1204A'}, {'Name':'C-2704 2nd Stage Equipment', 'Vicinity':'PID-1204B'},{'Name':'C-2704 3rd Stage Equipment', 'Vicinity':'PID-1204C'},{'Name':'C-2704 AJW Cooling System', 'Vicinity':'PID-1204K'},{'Name':'C-2704 Compressor Lubricant Equipment', 'Vicinity':'PID-1204E'},{'Name':'C-2704 Cooler Equipment', 'Vicinity':'PID-1204D'},{'Name':'C-2704 Cooler Equipment', 'Vicinity':'PID-1204J'},{'Name':'C-2704 Engine Lubricant Equipment', 'Vicinity':'PID-1204G'},{'Name':'C-2704 Start/Fuel Gas System', 'Vicinity':'PID-1204H'}]},{'Skid':'C-2705 Inlet Compressor Skid','Internals':[{'Name':'C-2705 1st Stage Equipment', 'Vicinity':'PID-1205A'}, {'Name':'C-2705 2nd Stage Equipment', 'Vicinity':'PID-1205B'},{'Name':'C-2705 3rd Stage Equipment', 'Vicinity':'PID-1205C'},{'Name':'C-2705 AJW Cooling System', 'Vicinity':'PID-1205K'},{'Name':'C-2705 Compressor Lubricant Equipment', 'Vicinity':'PID-1205E'},{'Name':'C-2705 Cooler Equipment', 'Vicinity':'PID-1205D'},{'Name':'C-2705 Cooler Equipment', 'Vicinity':'PID-1205J'},{'Name':'C-2705 Engine Lubricant Equipment', 'Vicinity':'PID-1205G'},{'Name':'C-2705 Start/Fuel Gas System', 'Vicinity':'PID-1205H'}]}]}, 
                'Instrument Air Area':{'Name':'Instrument Air Area','Vicinity':'PID-1700','Subarea':None,'Skid':None,'Piping':{'Name':'Instrument Air Area Piping','Vicinity':'PID-1700'},'Compression':None}, 
                'Outlet Area':{'Name':'Outlet Area','Vicinity':'PID-1400','Subarea':None,'Skid':None,'Piping':{'Name':'Outlet Area Piping','Vicinity':'PID-1400'},'Compression':None}, 
                'Process Area':{'Name':'Process Area','Vicinity':'PID-1300','Subarea':[{'Name':'Block Mounted Process Area','Vicinity':'PID-1300'},{'Name':'Heater Treater Area','Vicinity':'PID-1300'},{'Name':'Lower Process Area #1','Vicinity':'PID-1300'},{'Name':'Lower Process Area #2','Vicinity':'PID-1300'},{'Name':'Methanol Storage Area','Vicinity':'PID-1300'},{'Name':'Overhead Compression Area','Vicinity':'PID-1300'},{'Name':'Upper Process Area','Vicinity':'PID-1300'},],'Skid':[{'Name':'C-3701 Overhead Compressor Skid','Vicinity':'PID-1321'},{'Name':'C-3702 Overhead Compressor Skid','Vicinity':'PID-1322'}],'Piping':{'Name':'Process Area Piping','Vicinity':'PID-1300'},'Compression':[{'Skid':'C-3701 Overhead Compressor Skid','Internals':[{'Name':'C-3711 Cylinders & Cooler','Vicinity':'PID-1321B'},{'Name':'C-3711 Drive Train & Major Equipment','Vicinity':'PID-1321C'},{'Name':'C-3711 Lubrication','Vicinity':'PID-1321D'},{'Name':'C-3711 Suction Scrubber','Vicinity':'PID-1321A'}]},{'Skid':'C-3702 Overhead Compressor Skid','Internals':[{'Name':'C-3712 Cylinders & Cooler','Vicinity':'PID-1322B'},{'Name':'C-3712 Drive Train & Major Equipment','Vicinity':'PID-1322C'},{'Name':'C-3712 Lubrication','Vicinity':'PID-1322D'},{'Name':'C-3712 Suction Scrubber','Vicinity':'PID-1322A'}]}]}, 
                'Product Storage Area':{'Name':'Product Storage Area','Vicinity':'PID-1900','Subarea':None,'Skid':[{'Name':'NGL Loading Pump Skid','Vicinity':'PID-1902'},{'Name':'NGL Reinjection Pump Skid','Vicinity':'PID-1910'},{'Name':'Pipeline Oil Pump Skid','Vicinity':'PID-1911'}],'Piping':{'Name':'Product Storage Area Piping','Vicinity':'PID-1900'},'Compression':None}, 
                'Refrigeration Area':{'Name':'Refrigeration Area','Vicinity':'PID-1600','Subarea':[{'Name':'Refrigeration Compression Area','Vicinity':'PID-1600'},{'Name':'Refrigeration Condenser Area','Vicinity':'PID-1604'},{'Name':'Refrigeration Receiver Area','Vicinity':'PID-1600'},],'Skid':[{'Name':'C-6701 Refrigeration Compressor Skid','Vicinity':'PID-1601'},{'Name':'C-6702 Refrigeration Compressor Skid','Vicinity':'PID-1602'}],'Piping':{'Name':'Refrigeration Area Piping','Vicinity':'PID-1600'},'Compression':[{'Skid':'C-6701 Refrigeration Compressor Skid','Internals':[{'Name':'C-6701 Drive Train','Vicinity':'PID-1601C'},{'Name':'C-6701 E.J. Water System','Vicinity':'PID-1601F'},{'Name':'C-6701 Engine Oil','Vicinity':'PID-1601D'},{'Name':'C-6701 Fuel & Start Gas','Vicinity':'PID-1601E'},{'Name':'C-6701 Lube Oil System','Vicinity':'PID-1601B'},{'Name':'C-6701 Module Compressor','Vicinity':'PID-1601A'}]},{'Skid':'C-6702 Refrigeration Compressor Skid','Internals':[{'Name':'C-6702 Drive Train','Vicinity':'PID-1602C'},{'Name':'C-6702 E.J. Water System','Vicinity':'PID-1602F'},{'Name':'C-6702 Engine Oil','Vicinity':'PID-1602D'},{'Name':'C-6702 Fuel & Start Gas','Vicinity':'PID-1602E'},{'Name':'C-6702 Lube Oil System','Vicinity':'PID-1602B'},{'Name':'C-6702 Module Compressor','Vicinity':'PID-1602A'}]}]}, 
                'Site Area':{'Name':'Site Area','Vicinity':None,'Subarea':[{'Name':'Control Room','Vicinity':None},{'Name':'Eye Wash Stations','Vicinity':None},{'Name':'Fire Extinguishers','Vicinity':None}],'Skid':None,'Piping':None,'Compression':None}, 
                'Storage Tank Area':{'Name':'Storage Tank Area','Vicinity':'PID-1200','Subarea':None,'Skid':None,'Piping':None,'Compression':None}, 
                'Utilities Area':{'Name':'Utilities Area','Vicinity':'PID-1500','Subarea':[{'Name':'EG Regeneration Area','Vicinity':'PID-1500'},{'Name':'Hot Oil Heater Area','Vicinity':'PID-1500'},{'Name':'TEG Regeneration Area','Vicinity':'PID-1500'}],'Skid':None,'Piping':{'Name':'Utilities Area Piping','Vicinity':'PID-1500'},'Compression':None}}

### *Add Areas to 'SITE':*

In [117]:
areas = list(area_configs.keys())
hub_name = 'Test Hub'
hub_id = {'ID':'TEST', 'PID': 'TEST'}

In [114]:
all_areas = []   
for area in areas:
    template = {}
    template['ID'] = hub_id['ID']+'-'+area.replace('Area', '').upper()
    template['Name'] = hub_name.replace('Hub', '')+area
    template['IsLocation'] = True
    if vicinities.get(area) != None:
        template['Vicinity'] = hub_id['PID']+'-'+area_configs[area].get('Vicinity')
    template['Icon'] = 'iconlib/CL/symbols/worldxp_g.gif'
    template['ParentRef'] = {'ID': hub_id['ID']}
    template['ClassificationRef'] = {'ID': 'AREA'}
    template['RepairCenterRef'] = None
    all_areas.append(copy.deepcopy(template))

In [116]:
all_areas

In [32]:
for area in all_areas:
    _resp = _post('assets',[area])

### *Add Sub Areas to Each 'Area':*

In [25]:
all_sub_areas = []
for area in areas:
    subarea = area_configs[area].get('Subarea')
    if subarea != None:
        for sub in subarea:
            template = {}
            template['ID'] = hub_id['ID']+'-'+sub.get('Name').upper()
            template['Name'] = sub.get('Name')
            template['IsLocation'] = True
            template['Vicinity'] = hub_id['PID']+'-'+sub.get('Vicinity')
            if sub == 'Control Room':
                template['Icon'] = 'iconlib/AS/locations/housegreen_g.gif'
            elif sub == 'Eye Wash Stations':
                template['Icon'] = 'iconlib/AS/medical/eyeball_g.gif'
            elif sub == 'Fire Extinguishers':
                template['Icon'] = 'iconlib/AS/equipment/fireex_g.gif'  
            else:
                template['Icon'] = 'iconlib/CL/rooms/monitorxp2_g.gif'
            template['ParentRef'] = {'ID': hub_id['ID']+'-'+area.replace('Area','').upper()}
            template['ClassificationRef'] = {'ID': 'SUBAREA'}
            template['RepairCenterRef'] = None
            all_sub_areas.append(copy.deepcopy(template))

In [34]:
all_sub_areas

In [35]:
for sub in all_sub_areas:
    _resp = _post('assets',[sub])

### *Add Skids to Each 'Area':*

In [47]:
all_skids = []
for area in areas:
    skid = area_configs[area].get('Skid')
    if skid != None:
        for x in skid:
            template = {}
            template['ID'] = hub_id['ID']+'-'+x.get('Name').upper()
            template['Name'] = x.get('Name')
            template['IsLocation'] = True
            template['Vicinity'] = hub_id['PID']+'-'+x.get('Vicinity')
            template['Icon'] = 'iconlib/CL/rooms/room1_g.gif'
            template['ParentRef'] = {'ID': hub_id['ID']+'-'+area.replace('Area','').upper()}
            template['ClassificationRef'] = {'ID': 'SK'}
            template['RepairCenterRef'] = None
            all_skids.append(copy.deepcopy(template))

In [64]:
all_skids

In [50]:
for skid in all_skids:
    _resp = _post('assets',[skid])

### *Add Piping to Each 'Area':*

In [65]:
all_piping = []
for area in areas:
    pipe = area_configs[area].get('Piping')
    if pipe != None:
        template = {}
        template['ID'] = hub_id['ID']+'-'+pipe.get('Name').upper()
        template['Name'] = pipe.get('Name')
        template['IsLocation'] = True
        template['Vicinity'] = hub_id['PID']+'-'+pipe.get('Vicinity')
        template['Icon'] = 'iconlib/CL/equipment/conveyoricon_g.gif'
        template['ParentRef'] = {'ID': hub_id['ID']+'-'+area.replace('Area','').upper()}
        template['ClassificationRef'] = {'ID': 'PIPING'}
        template['RepairCenterRef'] = None
        all_piping.append(copy.deepcopy(template))

In [69]:
all_piping

In [68]:
for pipe in all_piping:
    _resp = _post('assets',[pipe])

### *Add Compressor Internal Areas to Each 'Skid':*

In [147]:
all_comp_internals = []
for area in areas:
    compression = area_configs[area].get('Compression')
    if compression != None:
        compressors = compression[0]
        internals = compressors.get('Internals')
        for i in internals: 
            template = {}
            template['ID'] = hub_id['ID']+'-'+i.get('Name').upper()
            template['Name'] = i.get('Name')
            template['IsLocation'] = True
            template['Vicinity'] = hub_id['PID']+'-'+i.get('Vicinity')
            template['Icon'] = 'iconlib/CL/classifications/components_g.gif'
            template['ParentRef'] = {'ID': hub_id['ID']+'-'+compressors.get('Skid').upper()}
            template['ClassificationRef'] = {'ID': 'COMPRESSOR INTERNALS'}
            template['RepairCenterRef'] = {'ID':'SA-HUBS-'+hub_id['ID']}
            template['ShoprRef'] = {'ID':'SA-HUBS-'+hub_id['ID']+'-MECH'}
            all_comp_internals.append(copy.deepcopy(template))

In [149]:
all_comp_internals

In [97]:
for internals in all_comp_internals:
    _resp = _post('assets',[internals])

### *Pull All Tags From _file and Add New Classifications to class_configs:*

In [264]:
class_configs = {'AE': {'Classification': 'AE',
  'Name': 'Analyzer Primary Element',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/ballxp_g.gif',
  'ShopRef': 'IE'},
 'AIH': {'Classification': 'AI',
  'Name': 'High Analysis Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/lightonit_g.gif',
  'ShopRef': 'IE'},
 'AIHH': {'Classification': 'AI',
  'Name': 'High Analysis Shutdown Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/lightonit_g.gif',
  'ShopRef': 'IE'},
 'AIT': {'Classification': 'AIT',
  'Name': 'Analysis Indicating Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/comp_light_g.gif',
  'ShopRef': 'IE'},
 'C': {'Classification': 'C',
  'Name': 'Compressor',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/gear_g.gif',
  'ShopRef': 'MECH'},
 'EFM': {'Classification': 'EFM',
  'Name': 'DP/Flow Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/equipmentwireless_g.gif',
  'ShopRef': 'IE'},
 'FE': {'Classification': 'FE',
  'Name': 'Flow Primary Element',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/ballxp_g.gif',
  'ShopRef': 'IE'},
 'FI': {'Classification': 'FI',
  'Name': 'Flow Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/symbols/caccount_g.gif',
  'ShopRef': 'IE'},
 'FIT': {'Classification': 'FIT',
  'Name': 'Flow Indicating Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/comp_light_g.gif',
  'ShopRef': 'IE'},
 'FT': {'Classification': 'FT',
  'Name': 'Flow Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/equipmentwireless_g.gif',
  'ShopRef': 'IE'},
 'HOA': {'Classification': 'HOA',
  'Name': 'Hand Off Auto Switch',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/cube_g.gif',
  'ShopRef': 'IE'},
 'LG': {'Classification': 'LG',
  'Name': 'Level Viewing Glass',
  'IsLocation': False,
  'Icon': 'iconlib/CL/grounds/swings_g.gif',
  'ShopRef': 'IE'},
 'LI': {'Classification': 'LI',
  'Name': 'Level Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/leversxp_g.gif',
  'ShopRef': 'IE'},
 'LIC': {'Classification': 'LIC',
  'Name': 'Level Indicating Controller',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/networkxp_g.gif',
  'ShopRef': 'IE'},
 'LIT': {'Classification': 'LIT',
  'Name': 'Level Indicating Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/comp_light_g.gif',
  'ShopRef': 'IE'},
 'LT': {'Classification': 'LT',
  'Name': 'Level Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/equipmentwireless_g.gif',
  'ShopRef': 'IE'},
 'P': {'Classification': 'P',
  'Name': 'Pump',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/equip2_g.gif',
  'ShopRef': 'MECH'},
 'PCV': {'Classification': 'PCV',
  'Name': 'Pressure Control Valve',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/funnel_g.gif',
  'ShopRef': 'IE'},
 'PI': {'Classification': 'PI',
  'Name': 'Pressure Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/meter1_g.gif',
  'ShopRef': 'IE'},
 'PIC': {'Classification': 'PIC',
  'Name': 'Pressure Indicating Controller',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/networkxp_g.gif',
  'ShopRef': 'IE'},
 'PIT': {'Classification': 'PIT',
  'Name': 'Pressure Indicating Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/comp_light_g.gif',
  'ShopRef': 'IE'},
 'PSV': {'Classification': 'PSV',
  'Name': 'Pressure Relief Valve',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/bellsystem_g.gif',
  'ShopRef': 'IE'},
 'PV': {'Classification': 'PV-INSTRUMENT',
  'Name': 'Instrument Regulator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/funnel_g.gif',
  'ShopRef': 'IE'},
 'PY': {'Classification': 'PY',
  'Name': 'Pressure Converter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/equipment/yaley_g.gif',
  'ShopRef': 'IE'},
 'QA': {'Classification': 'QA',
  'Name': 'Equipment Status Relay Alarm',
  'IsLocation': False,
  'Icon': 'iconlib/CL/symbols/explaination2_g.gif',
  'ShopRef': 'IE'},
 'QS': {'Classification': 'QS',
  'Name': 'Quantity/Event Switch',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/leverbigxp_g.gif',
  'ShopRef': 'IE'},
 'RO': {'Classification': 'RO',
  'Name': 'Restriction Orifice',
  'IsLocation': False,
  'Icon': 'iconlib/CL/custom1/ocs_g.gif',
  'ShopRef': 'IE'},
 'SV': {'Classification': 'SV',
  'Name': 'Solenoid Valve',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/funnel_g.gif',
  'ShopRef': 'IE'},
 'TE': {'Classification': 'TE',
  'Name': 'Temperature Primary Element',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/ballxp_g.gif',
  'ShopRef': 'IE'},
 'TI': {'Classification': 'TI',
  'Name': 'Temperature Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/medical/thermometer%202_g.gif',
  'ShopRef': 'IE'},
 'TK': {'Classification': 'TK',
  'Name': 'Tank',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/cylindar2_g.gif',
  'ShopRef': 'IE'},
 'TW': {'Classification': 'TW',
  'Name': 'Thermowell',
  'IsLocation': False,
  'Icon': 'iconlib/CL/medical/thermometer%203_g.gif',
  'ShopRef': 'IE'},
 'V': {'Classification': 'V',
  'Name': 'Vessel',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/cylindar_g.gif',
  'ShopRef': 'OPS'},
 'XS': {'Classification': 'XS',
  'Name': 'Unassigned Switch',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/leverbigxp_g.gif',
  'ShopRef': 'IE'},
 'XV': {'Classification': 'XV',
  'Name': 'Isolation Valve',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/funnel_g.gif',
  'ShopRef': 'IE'}}

In [186]:
current_classes = list(class_configs.keys())

In [255]:
pid_1201 = _file

In [256]:
instruments = []
for tag,number in zip(pid_1201[0],pid_1201[1]):
    instruments.append(tag.strip("''")+'-'+number.strip("''"))
len(instruments)

36

In [155]:
instruments

In [257]:
classifications = []
for item in instruments:
    classifications.append(item.split('-')[0])

In [258]:
list(set(classifications))

['PV',
 'EFM',
 'FE',
 'TE',
 'TW',
 'V',
 'C',
 'AIHH',
 'PI',
 'RO',
 'AIT',
 'QS',
 'LG',
 'FIT',
 'XV',
 'PSV',
 'AIH',
 'PCV',
 'SV',
 'FI',
 'XS',
 'QA',
 'TI',
 'PY',
 'AE']

In [259]:
classes_to_add = []
for _class in classifications:
    if _class not in current_classes:
        classes_to_add.append(_class)
list(set(classes_to_add))

['RO', 'XV']

<p style="color:red;">- ADD new classifications to 'class_configs' here</p>

In [262]:
class_configs['RO'] = {'Classification':'RO','Name':'Restriction Orifice','IsLocation':False,'Icon':'iconlib/CL/custom1/ocs_g.gif','ShopRef':'IE'}
class_configs['XV'] = {'Classification':'XV','Name':'Isolation Valve','IsLocation':False,'Icon':'iconlib/CL/classifications/funnel_g.gif','ShopRef':'IE'}
class_configs['HOA'] = {'Classification':'HOA','Name':'Hand Off Auto Switch','IsLocation':False,'Icon':'iconlib/CL/classifications/cube_g.gif','ShopRef':'IE'}

In [263]:
sorted_classes = dict(sorted(class_configs.items()))
sorted_classes

{'AE': {'Classification': 'AE',
  'Name': 'Analyzer Primary Element',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/ballxp_g.gif',
  'ShopRef': 'IE'},
 'AIH': {'Classification': 'AI',
  'Name': 'High Analysis Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/lightonit_g.gif',
  'ShopRef': 'IE'},
 'AIHH': {'Classification': 'AI',
  'Name': 'High Analysis Shutdown Indicator',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/lightonit_g.gif',
  'ShopRef': 'IE'},
 'AIT': {'Classification': 'AIT',
  'Name': 'Analysis Indicating Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/technology/comp_light_g.gif',
  'ShopRef': 'IE'},
 'C': {'Classification': 'C',
  'Name': 'Compressor',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/gear_g.gif',
  'ShopRef': 'MECH'},
 'EFM': {'Classification': 'EFM',
  'Name': 'DP/Flow Transmitter',
  'IsLocation': False,
  'Icon': 'iconlib/CL/classifications/equipmentwireless_g.gif',
  'ShopR

In [160]:
# if classs not in class_configs, then append to class_configs and sort again

### *Add Asstets to MC:*

In [187]:
def asset_builder(hub, tag, configs, page, parent_ref):
    template = {}
    template['ID'] = str(hub)+'-'+str(tag)
    template['Name'] = tag+' '+str(configs.get('Name'))
    template['IsLocation'] = configs.get('IsLocation')
    template['Vicinity'] = page
    template['Icon'] = configs.get('Icon')
    template['UDFChar9'] = str(hub)+'-'+str(tag)
    template['ParentRef'] = {'ID': parent_ref}
    template['ClassificationRef'] = {'ID': configs.get('Classification')}
    template['RepairCenterRef'] = {'ID': 'SA-HUBS-'+hub}
    template['ShopRef'] = {'ID': 'SA-HUBS-'+hub+'-'+configs.get('ShopRef')}
    return template

In [265]:
hub = 'SMI'
page = 'SMLY-PID-1205'
parent = 'SMI-C-2705 Inlet Compressor Skid'

In [266]:
assets = []
for item in instruments:
    key = item.split('-')[0]
    asset = asset_builder(hub, item, class_configs[key], page, parent)
    assets.append(copy.deepcopy(asset))
assets

[{'ID': 'SMI-XS-2705',
  'Name': 'XS-2705 Unassigned Switch',
  'IsLocation': False,
  'Vicinity': 'SMLY-PID-1205',
  'Icon': 'iconlib/CL/classifications/leverbigxp_g.gif',
  'UDFChar9': 'SMI-XS-2705',
  'ParentRef': {'ID': 'SMI-C-2705 Inlet Compressor Skid'},
  'ClassificationRef': {'ID': 'XS'},
  'RepairCenterRef': {'ID': 'SA-HUBS-SMI'},
  'ShopRef': {'ID': 'SA-HUBS-SMI-IE'}},
 {'ID': 'SMI-QA-2705',
  'Name': 'QA-2705 Equipment Status Relay Alarm',
  'IsLocation': False,
  'Vicinity': 'SMLY-PID-1205',
  'Icon': 'iconlib/CL/symbols/explaination2_g.gif',
  'UDFChar9': 'SMI-QA-2705',
  'ParentRef': {'ID': 'SMI-C-2705 Inlet Compressor Skid'},
  'ClassificationRef': {'ID': 'QA'},
  'RepairCenterRef': {'ID': 'SA-HUBS-SMI'},
  'ShopRef': {'ID': 'SA-HUBS-SMI-IE'}},
 {'ID': 'SMI-QS-2705',
  'Name': 'QS-2705 Quantity/Event Switch',
  'IsLocation': False,
  'Vicinity': 'SMLY-PID-1205',
  'Icon': 'iconlib/CL/classifications/leverbigxp_g.gif',
  'UDFChar9': 'SMI-QS-2705',
  'ParentRef': {'ID': 'S

In [267]:
existing_assets = []
for asset in assets:
    _id = asset.get('ID')
    _resp = _post('assets',[asset])
    try:
        if _resp['Results'][0]['ValidationResult'][0]['ValidationErrors'][0]['ErrorMessage'] == f'Asset {_id} conflicts with existing Asset.':
           existing_assets.append(copy.deepcopy(asset)) 
    except KeyError:
        pass

Connection NOT Successful
Status Code: 409

Connection Successful!
Status Code: 201
The asset has been added

Connection NOT Successful
Status Code: 409

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection NOT Successful
Status Code: 409

Connection Successful!
Status Code: 201
The asset has been added

Connection Successful!
Status Code: 201
The asset has been added

Connection

In [204]:
existing_assets

### *Edit Existing Assets:*

In [268]:
assets_to_update = []
for asset in existing_assets:
    _resp = _get(module='assets', _filter='ID', operator='eq', identifier=asset['ID'], top=500, skip=0)
    template = {}
    template['PK'] = _resp[0].get('PK')
    template['ID'] = asset.get('ID')
    template['Name'] = asset.get('Name')
    template['IsLocation'] = asset.get('IsLocation')
    template['Vicinity'] = asset.get('Vicinity')
    template['Icon'] = asset.get('Icon')
    template['UDFChar9'] = asset.get('UDFChar9')
    template['ParentRef'] = {'ID': asset['ParentRef'].get('ID')}
    template['ClassificationRef'] = {'ID': asset['ClassificationRef'].get('ID')}
    template['RepairCenterRef'] = asset.get('RepairCenterRef')
    template['ShopRef'] = asset.get('ShopRef')
    assets_to_update.append(copy.deepcopy(template))

Connection Successful!
Status Code: 200

Connection Successful!
Status Code: 200

Connection Successful!
Status Code: 200

Connection Successful!
Status Code: 200

Connection Successful!
Status Code: 200

Connection Successful!
Status Code: 200



In [215]:
assets_to_update

In [269]:
for item in assets_to_update:
    _resp = _put('assets',[item])

Connection Successful!
Status Code: 201
The asset has been updated

Connection Successful!
Status Code: 201
The asset has been updated

Connection Successful!
Status Code: 201
The asset has been updated

Connection Successful!
Status Code: 201
The asset has been updated

Connection Successful!
Status Code: 201
The asset has been updated

Connection Successful!
Status Code: 201
The asset has been updated

